### Create dictionary

In [1]:
from counter import doc_iterator
from gensim import corpora
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import pickle
import nltk
import regex as re
from pprint import pprint

In [2]:
jstor_zip_base = "C:/Users/benja/OneDrive - Cornell University/Desktop/soc_econ/data"

In [3]:
def get_di():
    return doc_iterator(
        jstor_zip_base,
        SKIP_N=99, # samples from entire corpus, takes 1 out of 100 documents
    )

In [7]:
def text_iterator(it):
    for i,(doi, doc) in enumerate(it):
        if i%1000 == 0:
            print(f"Document {i}")
        # if i == 0: print(doc['content'])
        yield [item for item in re.findall( "[A-Za-z]+", remove_stopwords(doc['content'].lower()) ) if len(item)>3] # yield function only temporarily loads the documents into storage

In [8]:
di = get_di()

In [9]:
dictionary = corpora.Dictionary( text_iterator(di) )

Found files drive-download-20211101T153841Z-001.zip, receipt-id-1215891-part-001.zip, receipt-id-1383671-part-001.zip, receipt-id-1383681-part-001.zip, receipt-id-1383691-part-001.zip
Iterating over  248384 documents
Document 0


In [10]:
outfile = open('gensimdic.pkl', 'wb')
pickle.dump(dictionary, outfile)                     
outfile.close()

### Run LDA

In [11]:
infile = open('gensimdic.pkl','rb')
dictionary = pickle.load(infile)
infile.close()

In [12]:
class MyCorpus(object):
    def __init__(self):
        self.di = get_di()        
        self.ti = text_iterator(self.di)
    def __iter__(self):
        yield dictionary.doc2bow( next(self.ti) )

In [13]:
# Run LDA by document and update the parameters
corpus = MyCorpus()
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                      id2word=dictionary,
                                      num_topics=10,
                                      update_every=10,
                                      chunksize=10000,
                                      passes=10)

Found files receipt-id-1215891-part-001.zip, receipt-id-1383671-part-001.zip, receipt-id-1383681-part-001.zip, receipt-id-1383691-part-001.zip
Iterating over  248380 documents
Document 0


In [14]:
for topic_id in range(lda.num_topics):
    topk = lda.show_topic(topic_id, 10)
    topk_words = [ w for w, _ in topk ]
    
    print('{}: {}'.format(topic_id, ' '.join(topk_words)))

0: data social research study theory effects family differences analysis studies
1: rockefeller social bulmer martin ruml ssrc research philanthropy council family
2: martin social rockefeller ruml research bulmer philanthropy council foundations ssrc
3: rockefeller social ruml martin bulmer philanthropy council research ssrc evidence
4: rockefeller bulmer social martin research ruml foundations council ssrc sciences
5: rockefeller social research ruml martin bulmer philanthropy foundations evidence fisher
6: rockefeller ruml research bulmer social martin philanthropy ssrc council foundations
7: social bulmer rockefeller martin research ruml ssrc sciences council philanthropy
8: social rockefeller martin bulmer research ssrc ruml philanthropy council sciences
9: rockefeller social bulmer research ruml martin philanthropy council ssrc evidence
